<a href="https://colab.research.google.com/github/frstndrd/Monografia/blob/main/tcc_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip

In [ ]:
import pandas as pd
from datetime import timedelta

# Function to convert time format, considering 24+ hour times
def convert_time(t):
    if isinstance(t, str):  # Check if t is a string
        hours, minutes = map(int, t.split(':'))
        if hours >= 24:
            hours -= 24
            return timedelta(days=1, hours=hours, minutes=minutes)
        return timedelta(hours=hours, minutes=minutes)
    else:
        return timedelta(0)  # Handle invalid or missing values

# Load the tasks from a GitHub-hosted .trf file
url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_do_s2.trf' # 92
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_sa_s2.trf' # 121
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_du_s2.trf' # 153
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_do.trf' # 169
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_sa.trf' # 262
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_du.trf' # 384
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/teste.trf'


tasks = pd.read_csv(url, sep='\s+', skiprows=2)

# Convert start time and end time to timedeltas
tasks['ST'] = tasks['ST'].apply(convert_time)
tasks['ET'] = tasks['ET'].apply(convert_time)

# Print task data for debugging
print(tasks.head())

# Parameters (temporarily relaxed for debugging)
min_task_duration = timedelta(minutes=30)   # Temporarily reduce task duration
max_gap_between_tasks = timedelta(minutes=240)  # Increase gap between tasks
min_work_time = timedelta(minutes=60)  # Reduce minimum work time
min_short_gap = timedelta(minutes=15)  # Minimum gap for short shifts (4-6 hours)
min_long_gap = timedelta(minutes=30)  # Minimum gap for long shifts (over 6 hours)

max_overtime = timedelta(hours=2)
max_shift_duration = timedelta(hours=6, minutes=40)
shift_generation_limit = 100

overtime_reward = 0.01
idle_penalty = 0.05

# Shift-Pool generation function with only 1 or 2 tasks per shift
def generate_shift_pool(tasks):
    nW = len(tasks)
    shift_pool = []

    # Add each task as a single-task shift
    for i in range(nW):
        task_i = tasks.iloc[i]
        shift_pool.append((task_i['ID'],))  # Single-task shift

    # Now add only two-task shifts (no 3-task shifts in this part)
    for i in range(nW - 1):
        task_i = tasks.iloc[i]
        start_i = task_i['ST']
        end_i = task_i['ET']

        # Ensure task i meets the minimum task duration
        if end_i - start_i >= min_task_duration:
            for j in range(i + 1, nW):
                task_j = tasks.iloc[j]
                start_j = task_j['ST']
                end_j = task_j['ET']

                # Ensure task j meets the minimum task duration
                if end_j - start_j >= min_task_duration:
                    idle_time = start_j - end_i
                    total_work_time = (end_i - start_i) + (end_j - start_j)

                    if timedelta(minutes=0) < idle_time <= max_gap_between_tasks:
                        if total_work_time <= max_shift_duration:
                            shift_pool.append((task_i['ID'], task_j['ID']))  # Only add 2-task shifts
    return shift_pool

# Simple valid_shift for debugging
def valid_shift(task_i, task_j, idle_time, total_work_time):
    loc_i_end = task_i['EL']
    loc_j_start = task_j['SL']

    # Basic check: task_j should start after task_i ends
    if idle_time.total_seconds() > 0:
        # Temporarily skip the other conditions to debug
        return True

    return False

# Generate the shift pool
shift_pool = generate_shift_pool(tasks)
print("Shift Pool:", shift_pool)
print(f"Shift Pool Size: {len(shift_pool)}")

   ID              ST  SL              ET  EL  OC  VC Linha
0   1 0 days 04:45:00   0 0 days 06:50:00  13   0  28  302D
1   2 0 days 04:50:00   0 0 days 07:00:00  14   0  29  302A
2   3 0 days 05:15:00   0 0 days 07:10:00  11   0  30  302B
3   4 0 days 05:15:00   0 0 days 07:30:00  12   0  31  302C
4   5 0 days 05:20:00   0 0 days 07:40:00  14   0  32   370
Shift Pool: [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (84,), (85,), (86,), (87,), (88,), (89,), (90,), (91,),

In [ ]:
print(tasks)

    ID              ST  SL              ET  EL  OC  VC Linha
0    1 0 days 04:45:00   0 0 days 06:50:00  13   0  28  302D
1    2 0 days 04:50:00   0 0 days 07:00:00  14   0  29  302A
2    3 0 days 05:15:00   0 0 days 07:10:00  11   0  30  302B
3    4 0 days 05:15:00   0 0 days 07:30:00  12   0  31  302C
4    5 0 days 05:20:00   0 0 days 07:40:00  14   0  32   370
..  ..             ...  ..             ...  ..  ..  ..   ...
87  88 0 days 21:50:00  13 1 days 00:10:00   0   0  28  302D
88  89 0 days 22:00:00  14 1 days 00:40:00   0   0  35  302A
89  90 0 days 22:30:00  11 1 days 00:50:00   0   0  36  302B
90  91 0 days 22:40:00  14 1 days 01:15:00   0   0  37   370
91  92 0 days 23:00:00  13 1 days 01:30:00   0   0  34  302D

[92 rows x 8 columns]


In [ ]:
from mip import Model, xsum, minimize, OptimizationStatus, CONTINUOUS
import time
from datetime import timedelta

# Modify restricted master problem to minimize shifts, but encourage split shifts and overtime
def restricted_master_problem(shift_pool, n_tasks, silent=True, rel_gap=1e-6, overtime_reward=1.0, split_shift_reward=1.0):
    rmp = Model()

    # Decision variables (continuous between 0 and 1 for relaxation)
    shift_vars = [rmp.add_var(var_type=CONTINUOUS, lb=0, ub=1) for _ in range(len(shift_pool))]

    # Objective: Minimize the number of shifts, but encourage overtime and split shifts
    total_overtime = []
    total_idle_time = []
    split_shift_rewards = []

    for i, shift in enumerate(shift_pool):
        # Calculate work time, idle time, overtime, split shifts, and shift validity
        work_time, idle_time, overtime, is_split_shift, is_valid_shift = calculate_shift_metrics(shift, tasks, max_shift_duration, max_overtime)

        # Since this is the restricted master problem, assume all shifts are valid (generated valid shifts)
        total_overtime.append(overtime)
        total_idle_time.append(idle_time)
        split_shift_rewards.append(is_split_shift)  # 1 if it's a split shift, 0 otherwise

    # Objective: Minimize number of shifts, but encourage overtime and split shifts
    rmp.objective = minimize(
        xsum(shift_vars)  # Minimize the number of shifts
        - overtime_reward * xsum(shift_vars[i] * total_overtime[i] for i in range(len(shift_vars)))  # Encourage overtime
        - split_shift_reward * xsum(shift_vars[i] * split_shift_rewards[i] for i in range(len(shift_vars)))  # Encourage split shifts
    )

    # Constraints: Each task must be covered at least once
    task_constraints = {}
    for task in range(1, n_tasks + 1):
        task_constraints[task] = rmp.add_constr(
            xsum(shift_vars[i] for i, shift in enumerate(shift_pool) if task in shift) == 1
        )

    # Solve the RMP with a near-zero relative gap (for exact optimization)
    rmp.rel_gap = rel_gap
    status = rmp.optimize()

    if not silent:
        if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
            print(f"Final Objective function value: {rmp.objective_value}")
        else:
            print(f"Warning: No valid solution found (status: {status}).")

    # Collect dual variables and selected shifts for column generation
    if status in {OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE}:
        dual_vars = [task_constraints[task].pi if task_constraints[task].pi is not None else 0
                     for task in range(1, n_tasks + 1)]
        selected_shifts = [shift_pool[i] for i in range(len(shift_pool)) if shift_vars[i].x >= 0.5]
    else:
        return None, [], []

    return rmp, dual_vars, selected_shifts


# Function to calculate shift metrics: work time, idle time, overtime, and split shift reward
def calculate_shift_metrics(shift, tasks, max_shift_duration, max_overtime):
    total_work_time = timedelta(0)
    idle_time = timedelta(0)
    overtime = timedelta(0)
    is_split_shift = 0

    last_end_time = None

    for task_id in shift:
        task = tasks[tasks['ID'] == task_id].iloc[0]
        start = task['ST']
        end = task['ET']

        work_time = end - start
        total_work_time += work_time

        if last_end_time is not None:
            idle_period = start - last_end_time
            idle_time += idle_period

            # If idle time is greater than 2 hours, it's a split shift
            if idle_period > timedelta(hours=2):
                is_split_shift = 1

        last_end_time = end

    # Calculate overtime if total work time exceeds max shift duration
    total_paid_time = total_work_time + idle_time
    if total_paid_time > max_shift_duration:
        overtime = total_paid_time - max_shift_duration
        overtime = min(overtime, max_overtime)  # Cap overtime at max allowed overtime

    # Make sure total shift time does not exceed 8h40m
    total_shift_time = total_work_time + idle_time
    is_valid_shift = total_shift_time <= (max_shift_duration + max_overtime)

    # Return the times as hours (numeric values), split shift flag, and shift validity flag
    return total_work_time.total_seconds() / 3600, idle_time.total_seconds() / 3600, overtime.total_seconds() / 3600, is_split_shift, is_valid_shift



# Column generation process with enhanced debugging and manual optimality gap calculation
def column_generation(tasks, initial_shift_pool, rel_gap=1e-6):
    n_tasks = tasks['ID'].max()

    shift_pool = initial_shift_pool.copy()
    all_shifts = set(tuple(shift) for shift in shift_pool)
    final_shifts = []

    upper_bound = None
    lower_bound = None  # Will be set based on the first feasible solution
    iteration = 0

    while True:
        iteration += 1
        print(f"\nIteration {iteration}: Solving Restricted Master Problem...")

        # Measure the time taken to solve the RMP
        start_rmp_time = time.time()

        # Solve the RMP for the current iteration (silent mode)
        rmp, dual_vars, selected_shifts = restricted_master_problem(
            shift_pool, n_tasks, silent=True, rel_gap=rel_gap)

        end_rmp_time = time.time()
        rmp_time_taken = end_rmp_time - start_rmp_time
        print(f"Iteration {iteration}: RMP solved in {rmp_time_taken:.2f} seconds.")

        # If no valid solution, exit
        if rmp is None:
            print(f"No valid solution, skipping this iteration.")
            break

        # Capture the upper bound (objective value)
        upper_bound = rmp.objective_value

        # If this is the first iteration, use the current solution as a lower bound estimate
        if lower_bound is None:
            lower_bound = upper_bound

        # If optimality gap is small enough, stop early
        if rmp.gap is not None and rmp.gap < rel_gap:
            print(f"Objective function value = {upper_bound}")
            print(f"Optimality gap is below {rel_gap * 100:.2f}%. Stopping early.")
            final_shifts = selected_shifts
            break

        # Measure the time taken to generate new shifts
        start_pricing_time = time.time()

        # Solve the pricing problem to generate new shifts
        new_columns = pricing_problem(tasks, dual_vars, shift_pool)

        end_pricing_time = time.time()
        pricing_time_taken = end_pricing_time - start_pricing_time
        print(f"Iteration {iteration}: Pricing problem solved in {pricing_time_taken:.2f} seconds.")

        # Add new unique columns to the shift pool
        new_unique_columns = [tuple(shift) for shift in new_columns if tuple(shift) not in all_shifts]
        shift_pool += new_unique_columns
        all_shifts.update(new_unique_columns)

        # Stop if no new columns generated
        if not new_unique_columns:
            print(f"Iteration {iteration}: No new shifts generated. Stopping.")
            final_shifts = selected_shifts
            break

    # Final solve of the RMP to report final metrics
    print("\nFinal Solve of Restricted Master Problem...")
    rmp, dual_vars, final_shifts = restricted_master_problem(
        shift_pool, n_tasks, silent=False, rel_gap=rel_gap)

    # Print the shift details table (instead of the raw final shifts list)
    print_shift_details(final_shifts, tasks)

    return final_shifts




# Pricing problem to generate multi-task shifts, including combining existing multi-task shifts
def pricing_problem(tasks, dual_vars, shift_pool):
    new_columns = []
    nW = len(tasks)

    # Debug: Track how many shifts are checked and how many become valid columns
    total_checked = 0
    total_valid = 0

    # Single-task shifts
    for i in range(nW):
        task_i = tasks.iloc[i]
        new_shift = (task_i['ID'],)
        reduced_cost = calculate_reduced_cost(new_shift, dual_vars)

        total_checked += 1  # Count how many shifts are checked

        # Validate if shift is within allowed time using calculate_shift_metrics
        work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(new_shift, tasks, max_shift_duration, max_overtime)

        # Skip bad shifts that exceed 8h40m total time
        if reduced_cost < 0 and is_valid_shift:
            new_columns.append(new_shift)
            total_valid += 1  # Count valid shifts added to columns

    # Two-task shifts
    for i in range(nW - 1):
        task_i = tasks.iloc[i]
        end_i = task_i['ET']

        for j in range(i + 1, nW):
            task_j = tasks.iloc[j]
            start_j = task_j['ST']
            end_j = task_j['ET']

            idle_time = start_j - end_i
            total_work_time = (end_i - task_i['ST']) + (end_j - start_j)

            if valid_shift(task_i, task_j, idle_time, total_work_time):
                new_shift = (task_i['ID'], task_j['ID'])
                reduced_cost = calculate_reduced_cost(new_shift, dual_vars)

                total_checked += 1  # Count how many shifts are checked

                # Validate if shift is within allowed time
                work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(new_shift, tasks, max_shift_duration, max_overtime)

                # Skip bad shifts that exceed 8h40m total time
                if reduced_cost < 0 and is_valid_shift:
                    new_columns.append(new_shift)
                    total_valid += 1  # Count valid shifts added to columns

    # Combine existing multi-task shifts with additional tasks
    for shift in shift_pool:
        if len(shift) == 2:  # Only consider two-task shifts for extension
            task_i_id, task_j_id = shift

            task_i = tasks[tasks['ID'] == task_i_id].iloc[0]
            task_j = tasks[tasks['ID'] == task_j_id].iloc[0]
            end_j = task_j['ET']
            loc_j = task_j['EL']

            for k in range(nW):
                task_k = tasks.iloc[k]
                start_k = task_k['ST']
                end_k = task_k['ET']
                loc_k = task_k['SL']

                idle_time = start_k - end_j
                total_work_time = (end_j - task_i['ST']) + (end_k - start_k)

                if valid_shift(task_j, task_k, idle_time, total_work_time):
                    # Combine the existing shift (1-3) with task 5
                    extended_shift = (*shift, task_k['ID'])
                    reduced_cost = calculate_reduced_cost(extended_shift, dual_vars)

                    total_checked += 1  # Count how many shifts are checked

                    # Validate if shift is within allowed time
                    work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(extended_shift, tasks, max_shift_duration, max_overtime)

                    # Skip bad shifts that exceed 8h40m total time
                    if reduced_cost < 0 and is_valid_shift:
                        new_columns.append(extended_shift)
                        total_valid += 1  # Count valid shifts added to columns

    # Debug: Print how many shifts were checked and how many were valid columns
    print(f"Pricing Problem: {total_checked} shifts checked, {total_valid} valid columns generated.")

    return new_columns





# Function to calculate reduced cost based on dual variables
def calculate_reduced_cost(shift, dual_vars):
    reduced_cost = sum(dual_vars[task - 1] for task in shift) - 1
    return reduced_cost


# Function to format timedelta values for printing
def format_timedelta(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    if days == 0:
        return f"0 days {hours:02}:{minutes:02}:{seconds:02}"
    else:
        return f"{days} days {hours:02}:{minutes:02}:{seconds:02}"


# Updated function to print shift details and track split shifts and total overtime
def print_shift_details(final_shifts, tasks):
    assigned_tasks = set()  # To ensure each task is assigned only once
    split_shift_count = 0   # To count the number of split shifts
    total_overtime = timedelta(0)  # To track total overtime

    print(f"\n{'Shift':<10}\t{'Work Time':<25}{'Idle Time':<25}{'Overtime':<25}{'Split Shift?':<15}")
    print("=" * 100)

    for shift in final_shifts:
        if not shift or any(task_id in assigned_tasks for task_id in shift):  # Skip empty or already assigned shifts
            continue

        shift_tasks = []
        total_work_time = timedelta(0)
        last_end_time = None
        is_split_shift = False

        # Process each task in the shift
        for task_id in shift:
            if task_id in assigned_tasks:
                continue  # Skip if the task is already assigned to another shift

            task = tasks[tasks['ID'] == task_id].iloc[0]
            start = task['ST']
            end = task['ET']

            shift_tasks.append(task_id)
            work_time = end - start
            total_work_time += work_time

            assigned_tasks.add(task_id)  # Mark the task as assigned

            if last_end_time is not None:
                idle_time = start - last_end_time
                if idle_time > timedelta(hours=2):
                    is_split_shift = True  # Mark as a split shift

            last_end_time = end

        # If no tasks were assigned in this shift, skip
        if not shift_tasks:
            continue

        # Calculate idle time: any non-work time within the max shift duration
        total_paid_time = timedelta(hours=6, minutes=40)  # 6h40m shift limit
        idle_time = max(timedelta(0), total_paid_time - total_work_time)

        # Calculate overtime
        overtime = max(timedelta(0), total_work_time - max_shift_duration)
        overtime = min(overtime, max_overtime)  # Cap overtime

        # Add to total overtime
        total_overtime += overtime

        # Count split shifts
        if is_split_shift:
            split_shift_count += 1

        # Print details
        print(f"{'-'.join(map(str, shift_tasks))}\t\t{format_timedelta(total_work_time)}\t\t"
              f"{format_timedelta(idle_time)}\t\t{format_timedelta(overtime)}\t\t{'Yes' if is_split_shift else 'No'}")

    # Print the split shift count and total overtime
    print("\nSummary:")
    print(f"Total Split Shifts: {split_shift_count}")
    print(f"Total Overtime: {format_timedelta(total_overtime)}")


# Example: Run the column generation algorithm and evaluate the final solution
start_time = time.time()
final_shifts = column_generation(tasks, shift_pool)
total_time = time.time() - start_time
print(f"Column generation completed in {total_time:.2f} seconds.")


Iteration 1: Solving Restricted Master Problem...
Iteration 1: RMP solved in 3.51 seconds.
Pricing Problem: 40218 shifts checked, 4765 valid columns generated.
Iteration 1: Pricing problem solved in 118.26 seconds.

Iteration 2: Solving Restricted Master Problem...
Iteration 2: RMP solved in 10.10 seconds.
Pricing Problem: 41989 shifts checked, 4784 valid columns generated.
Iteration 2: Pricing problem solved in 113.69 seconds.

Iteration 3: Solving Restricted Master Problem...
Iteration 3: RMP solved in 8.82 seconds.
Pricing Problem: 41989 shifts checked, 4778 valid columns generated.
Iteration 3: Pricing problem solved in 114.86 seconds.
Iteration 3: No new shifts generated. Stopping.

Final Solve of Restricted Master Problem...
Final Objective function value: -58.66666666666667

Shift     	Work Time                Idle Time                Overtime                 Split Shift?   
4-32		0 days 04:50:00		0 days 01:50:00		0 days 00:00:00		Yes
5-30		0 days 05:00:00		0 days 01:40:00		0 d

In [ ]:
# !cat /proc/cpuinfo